<a href="https://colab.research.google.com/github/AmMasse/Family-GPT/blob/main/Family.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
!pip install datasets transformers

In [96]:
import transformers

print(transformers.__version__)

4.48.3


In [97]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "/content/train_data.jsonl", "validation": "/content/validation_data.jsonl"})

In [98]:
datasets["train"][10]

{'text': '{"question": "What was the relationship between Unknown and John?", "answer": "Unknown was the grandparent of John.", "context": {"id": 1, "name": "Unknown", "gender": "Unknown", "birth_date": "1900-01-01", "death_date": "1930-01-01", "birth_location": "Rwampara, UG", "occupation": "Blacksmith", "generation": "Alpha", "notes": "He is a Munyankole by Tribe. He is Generation Alpha, born roughly in the 1900s. There is hardly anything known about this generation. There wasnt any record keeping whatsoever. Not to mention, the children and grandchildren referred to their parents and grandparents as mama or baba; because, using their names was not seen as respectful. He died young. He was a flute player. He learnt to blacksmith from some Italians Iron Ore Miners near his home. He loved playing a flute and entertaining.", "spouses": [{"id": 2, "name": "Unknown"}], "parents": [], "children": [{"id": 9, "name": "Biryaberema Matthew"}], "siblings": [], "lifespan": 30}}'}

In [99]:
from datasets import ClassLabel
import random
import json
import pandas as pd
from IPython.display import display, HTML


In [100]:

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [101]:
show_random_elements(datasets["train"])

,text
0,"{""question"": ""How did Leah Abooki and Kabahuuma Akiki Malingo fit in the family tree?"", ""answer"": ""Leah Abooki was the grandparent of Kabahuuma Akiki Malingo."", ""context"": {""id"": 8, ""name"": ""Leah Abooki"", ""gender"": ""Female"", ""birth_date"": ""1912-04-04"", ""death_date"": ""1965-08-08"", ""birth_location"": ""Haibaale Ly'ekitinisa-Mwege"", ""occupation"": ""Homemaker"", ""generation"": ""Alpha"", ""notes"": ""She is a Mutooro by Tribe. A Musiita by Clan. Her name Leah, Abooki. She was from Haibaale Lyekitinisa (the sacred rock, Mwenge. She was a Musiita by clan. She gave birth to 5 children: Her children are: Akiki Suzan (1st born); Atwooki (2nd born); Atenyi (3rd Born); there was a forth; and lastly Eseri Bikanga Amooti (5th born)."", ""spouses"": [{""id"": 7, ""name"": ""Azalia Kakyomya Amooti""}], ""parents"": [], ""children"": [{""id"": 12, ""name"": ""Eseri Bikanga Amooti Mwirumubi""}], ""siblings"": [], ""lifespan"": 53}}"
1,"{""question"": ""What was the relationship between Kiiza Akiki and Tusiime Beatrice Atenyi?"", ""answer"": ""Kiiza Akiki and Tusiime Beatrice Atenyi were siblings."", ""context"": {""id"": 25, ""name"": ""Kiiza Akiki"", ""gender"": ""Male"", ""birth_date"": """", ""death_date"": ""2016-07-14"", ""birth_location"": ""Muhumbu"", ""occupation"": ""Teacher"", ""generation"": ""Gamma"", ""notes"": ""She studied education and taught in Kasese schools. She was struck with HIV and incapacitated for many years before dying in 2016."", ""spouses"": [], ""parents"": [{""id"": 11, ""name"": ""Yohanna Apuuli Mwirumubi""}, {""id"": 12, ""name"": ""Eseri Bikanga Amooti Mwirumubi""}], ""children"": [], ""siblings"": [{""id"": 20, ""name"": ""Araali""}, {""id"": 21, ""name"": ""Kabahuuma Akiki Malingo""}, {""id"": 22, ""name"": ""Atwooki""}, {""id"": 23, ""name"": ""Nyangoma Catherine Masse""}, {""id"": 24, ""name"": ""Nyakato Getrude Abooki""}, {""id"": 26, ""name"": ""Felle Akugizibwe Adyeri""}, {""id"": 27, ""name"": ""Musinguzi Anatoli Apuuli""}, {""id"": 28, ""name"": ""Ngozi Goretti""}, {""id"": 29, ""name"": ""Tusiime Beatrice Atenyi""}]}}"
2,"{""question"": ""Were Eseri Bikanga Amooti Mwirumubi and Marunga Sylivia related? If so, how?"", ""answer"": ""Eseri Bikanga Amooti Mwirumubi was the grandparent of Marunga Sylivia."", ""context"": {""id"": 12, ""name"": ""Eseri Bikanga Amooti Mwirumubi"", ""gender"": ""Female"", ""birth_date"": ""1950-09-09"", ""death_date"": ""2012-10-30"", ""birth_location"": ""Kyarusozi-Mwenge"", ""occupation"": ""Homemaker"", ""generation"": ""Beta"", ""notes"": ""Eseri Bikanga Amooti Mwirumubi, was a devoutly religious woman; who valued family and hardwork. She made sure to pass on those values to her children by encouraging them to work with her in all that she was doing. She was a Muchwezi by clan. A Mutooro by Tribe. She gave birth to 10 children with Yohanna Mwirumubi Apuuli. Her children are: her first born was a boy who died at birth; Akiki Kabahuuma Malingo (2nd born). Place of residence: Harubaho, with her husband Malingo, parents of 6 children; third born, a boy died at birth; fourth born: identical twins: Nyangoma Catherine, mother of 3 boys: Ayebale Deus, Amanyire Daniel, Ainembabazi John Dickson and Nyakato Getrude, mother of 5 children: Nyakojjo James, Musinguzi Robert, Kanyunyuzi Jacinta, Good Francis, Marunga; sixth born: Kiiza PatricK Akiki, passed on 4th July, 2016, without any children; seventh born: Felle Akugizibwe Adyeeri; a qualified teacher of English and SST, mother of one: Kabatooro Christine, she passed on July 2007; eighth born: Musinguzi Anatooli Apuuli; ninth born: Ngozi Goretti Atwooki, mother of three: Komuhendo Julliet, Muhirwa Lawrence, Kamukama Judith; 10th born: Tusiime Beatrice Atenyi, mother of 6:Sabiiti Collins, Kyakyo Akello Bridget, Bataala Peter Innocent, Kalege Andrew, Mbwiire Timothy, Kalogo Gerald. She passed on 15th Aug., 2016."", ""spouses"": [{""id"": 11, ""name"": ""Yohanna Apuuli Mwirumubi""}], ""parents

In [102]:
len(datasets["train"])

1159

In [103]:
len(datasets["validation"])

301

In [104]:
model_checkpoint = "distilgpt2"

In [105]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [106]:
tokenizer.pad_token = tokenizer.eos_token

In [107]:
def tokenize_function(examples):
    questions = []
    contexts = []

    for text in examples["text"]:
        try:
            data = json.loads(text)  # Parse JSON string

            # Extract question and context, handling missing keys
            question = data.get("question", "")
            context = data.get("context", "")

            # Ensure values are strings
            questions.append(str(question))
            contexts.append(str(context))

        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {e}")
            questions.append("")
            contexts.append("")

    return tokenizer(
        questions,
        contexts,
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

In [108]:
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"])

In [109]:
print(tokenized_datasets["train"][0])

{'input_ids': [2061, 466, 345, 760, 546, 16185, 30, 90, 6, 312, 10354, 352, 11, 705, 3672, 10354, 705, 20035, 3256, 705, 8388, 10354, 705, 20035, 3256, 705, 24280, 62, 4475, 10354, 705, 48104, 12, 486, 12, 486, 3256, 705, 22595, 62, 4475, 10354, 705, 1129, 1270, 12, 486, 12, 486, 3256, 705, 24280, 62, 24886, 10354, 705, 49, 86, 696, 3301, 11, 471, 38, 3256, 705, 19596, 341, 10354, 705, 3629, 4595, 22947, 3256, 705, 20158, 10354, 705, 38077, 3256, 705, 17815, 10354, 705, 1544, 318, 257, 12107, 88, 962, 2305, 416, 27804, 13, 679, 318, 16588, 12995, 11, 4642, 7323, 287, 262, 21489, 82, 13, 1318, 318, 8941, 1997, 1900, 546, 428, 5270, 13, 1318, 373, 429, 597, 1700, 5291, 16014, 13, 1892, 284, 3068, 11, 262, 1751, 290, 28986, 6412, 284, 511, 3397, 290, 28571, 355, 285, 1689, 393, 9289, 64, 26, 780, 11, 1262, 511, 3891, 373, 407, 1775, 355, 25923, 13, 679, 3724, 1862, 13, 679, 373, 257, 781, 1133, 2137, 13, 679, 26338, 284, 2042, 21453, 422, 617, 41720, 7931, 22452, 1855, 364, 1474, 465, 136

In [110]:
# block_size = tokenizer.model_max_length
block_size = 128

In [111]:
def group_texts(examples):
    # Ensure the examples have the expected keys
    if "input_ids" not in examples:
        raise KeyError("Key 'input_ids' not found in the tokenized dataset.")

    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples["input_ids"])

    # Drop the small remainder
    total_length = (total_length // block_size) * block_size

    # Split into chunks of block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }

    # Create labels
    result["labels"] = result["input_ids"].copy()

    return result

In [112]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/1159 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/301 [00:00<?, ? examples/s]

In [113]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

" and grandchildren referred to their parents and grandparents as mama or baba; because, using their names was not seen as respectful. He died young. He was a flute player. He learnt to blacksmith from some Italians Iron Ore Miners near his home. He loved playing a flute and entertaining.', 'spouses': [{'id': 2, 'name': 'Unknown'}], 'parents': [], 'children': [{'id': 9, 'name': 'Biryaberema Matthew'}], 'siblings': [], 'lifespan': 30}<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>"

In [114]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [115]:
from transformers import Trainer, TrainingArguments

In [116]:
model_name = model_checkpoint.split("/")[-1]  # Extract model name from checkpoint path

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned",
    evaluation_strategy="steps",
    eval_steps=1000,               # Adjust to less frequent evaluation
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,
    num_train_epochs=1,            # Start with 1 epoch for testing
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,             # Reduced logging frequency
    save_steps=1000,               # Reduced frequency of saving checkpoints
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    push_to_hub=False,             # Disable pushing to hub for testing
    report_to="none",              # Avoid reporting to external tools
    fp16=True                      # Enable mixed precision training
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [117]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [118]:
import wandb
wandb.init(mode= "disabled")

In [119]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
1000,0.237800,0.113365
2000,0.216900,0.093599


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=2318, training_loss=0.34030866262931264, metrics={'train_runtime': 237.7714, 'train_samples_per_second': 38.995, 'train_steps_per_second': 9.749, 'total_flos': 302842933936128.0, 'train_loss': 0.34030866262931264, 'epoch': 1.0})

In [121]:
import math

# Evaluate on the training dataset
train_results = trainer.evaluate(eval_dataset=lm_datasets["train"])
train_perplexity = math.exp(train_results['eval_loss'])
print(f"Training Perplexity: {train_perplexity:.2f}")

# Evaluate on the validation dataset
val_results = trainer.evaluate(eval_dataset=lm_datasets["validation"])
val_perplexity = math.exp(val_results['eval_loss'])
print(f"Validation Perplexity: {val_perplexity:.2f}")

Training Perplexity: 1.16
Validation Perplexity: 1.10


In [122]:
pip install huggingface_hub

In [126]:
from huggingface_hub import login

login()

In [127]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/amDANIEL2024/distilgpt2-finetuned/commit/5a68ada272c38310ae6b7f986478fab41362329c', commit_message='End of training', commit_description='', oid='5a68ada272c38310ae6b7f986478fab41362329c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/amDANIEL2024/distilgpt2-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='amDANIEL2024/distilgpt2-finetuned'), pr_revision=None, pr_num=None)